In [79]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import gamma

In [80]:
# Parameters 

h = 1/7
pho = 2/3
g = 2/3
R = 19
T = 7*R

In [86]:
def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [87]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [88]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]  
    M = theta[:16].reshape(4,4)
    betas = 1/8000 * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
    C = np.random.binomial(E, 1-np.exp(-h*pho)) 
    D = np.random.binomial(I, 1-np.exp(-h*g)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R + D
    X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
    return X_new

In [102]:
# Likelihood of y knowing theta

def log_f(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X = mutation(theta, X)
        C[i] = np.random.binomial(X[:, 1], 1-np.exp(-h*pho))
    l = np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5)
    if any(np.isnan(l)):
        print(y, np.sum(C, axis=0), theta[16:])
    return(np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5))

In [90]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, theta, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    w_init = np.zeros((N, 4))
    X_all = np.array([X for r in range(R+1)])
    for n in range(N):
        w_init[n] = log_f(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    X_all[1] = X
    for r in range(R):
        w_hats = np.zeros((N,4))
        for k in range(4):
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), size=N, p=W[:, k])
                w_hat = np.zeros(N)
            else:
                A = np.arange(0, N)
                w_hat = w[:, k]
            for n in range(N):
                X[n][k] = X[A[n]][k]
            w_hats[:, k] = w_hat
        for n in range(N):
            w[n] = w_hats[n]+log_f(y[r], theta, X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))
        X_all[r+1] = X
    return(X_all)

In [93]:
theta = np.concatenate((np.random.gamma(5, 1, 16), get_truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
X_all = BootstrapFilter(Y, theta, 999, 1000, R=19)

In [94]:
print(X_all[8])

[[[ 100  278  264  307]
  [ 248  485  388  569]
  [ 232  924  978 1333]
  [  85  505  534  771]]

 [[  69  271  251  358]
  [ 224  506  409  551]
  [ 250  974  940 1303]
  [  73  484  557  781]]

 [[ 107  284  230  328]
  [ 283  509  393  505]
  [ 335 1018  859 1255]
  [ 123  510  530  732]]

 ...

 [[ 109  276  243  321]
  [ 293  525  394  478]
  [ 367 1069  921 1110]
  [ 141  564  472  718]]

 [[  77  274  287  311]
  [ 219  490  453  528]
  [ 216  939  928 1384]
  [  79  503  515  798]]

 [[ 115  239  268  327]
  [ 261  514  385  530]
  [ 369 1081  937 1080]
  [ 146  570  514  665]]]


In [95]:
def log_prior(theta):
    prior = 1
    for i in range(16):
        prior = prior * gamma.pdf(theta[i], a=5, loc=0, scale=1)
    for i in range(4):
        prior = prior * truncnorm.pdf(theta[16+i], a=0, b=1, loc=0.5, scale=0.5)
    return np.log(1+prior)

In [96]:
def LogL(y, theta, X):
    L = np.zeros(4)
    for r in range(R):
        S = 0
        for n in range(len(X[r])):
            x = X[r][n]
            S = S + np.exp(log_f(y[r], theta, x)) - 1.5
            #if any(np.isnan(S)):
                #print(log_f(y[r], theta, x))
        for k in range(4):
            if S[k] == 0:
                L[k] = -
            L[k] = L[k] + np.log(S[k]+1) - len(X[r])
    return L+log_prior(theta)

In [100]:
def PMMH(y, ESS_min, Nx, Nt, R=19):
    theta = np.concatenate((np.random.gamma(5, 1, 16), get_truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)    
    thetas = np.array([theta for i in range(Nt)])
    X = BootstrapFilter(y, theta, ESS_min, Nx, R=19)
    L = LogL(y, theta, X)
    for n in range(1, Nt):
        theta_new =  np.concatenate((get_truncated_normal(mean=theta[:16], sd=0.1, low=0, upp=np.inf).rvs(), get_truncated_normal(mean=theta[16:], sd=0.1, low=0, upp=1).rvs()), axis=None)
        #print(theta_new)
        X_new = BootstrapFilter(y, theta_new, ESS_min, Nx, R=19)
        L_new = LogL(y, theta_new, X_new)
        v = L_new - L
        #print(L_new)
        for k in range(4):      
            u = np.random.uniform()
            #print(k, v[k])
            if np.log(u)<v[k]:
                L[k] = L_new[k]
                thetas[n][k:k+4] = theta_new[k:k+4]
                thetas[n][16+k] = theta_new[16+k]
                theta[k:k+4] = theta_new[k:k+4]
                theta[16+k] = theta_new[16+k]
            else:
                thetas[n][k:k+4] = theta[k:k+4]
                thetas[n][16+k] = theta[16+k]    
    return thetas

In [104]:
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
PMMH(Y, 999, 25, 10000, R=19)

KeyboardInterrupt: 